<a href="https://colab.research.google.com/github/sunyingjian/AI-in-well-logging/blob/master/catboost%E7%BE%8E%E5%9B%BD%E6%B2%B9%E7%94%B0_%E5%BB%B6%E5%AE%89%E6%B2%B9%E7%94%B0_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#延安油田

##引入数据

In [1]:
!git clone https://github.com/sunyingjian/numpy-.git

Cloning into 'numpy-'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 327 (delta 0), reused 0 (delta 0), pack-reused 324
Receiving objects: 100% (327/327), 181.81 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Checking out files: 100% (162/162), done.


In [2]:
!pip install catboost

     |████████████████████████████████| 66.2MB 115kB/s 


In [3]:
import pandas as pd
import numpy as np
import catboost as cb
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

In [4]:
train = pd.read_csv('/content/numpy-/3345train_data.csv')
test = pd.read_csv('/content/numpy-/3345test_data.csv')

In [5]:
train

,AC,CAL,GR,K,RD,SP,Core Lithology
0,0.029102,0.031789,0.026724,0.202335,0.078986,0.333754,5
1,0.032883,0.033469,0.030592,0.203141,0.076064,0.333669,5
2,0.034385,0.037006,0.032359,0.200121,0.074503,0.333619,5
3,0.243816,0.143381,0.086581,0.290115,0.017649,0.094025,3
4,0.248043,0.146786,0.084643,0.283874,0.017286,0.090554,3
...,...,...,...,...,...,...,...
3340,0.012092,0.067866,0.007657,0.077713,0.181798,0.348797,5
3341,0.011831,0.069989,0.007606,0.081135,0.206839,0.348560,5
3342,0.011533,0.070121,0.009591,0.081337,0.250115,0.348470,5
3343,0.008903,0.069325,0.010596,0.081135,0.281774,0.348340,5


In [6]:
train_data = train.drop(columns=['Core Lithology'])

In [7]:
train_label = train['Core Lithology'].values

In [8]:
test_data = test.drop(columns=['Core Lithology'])
test_label = test['Core Lithology'].values

##使用catboost

In [9]:
train_pool = Pool(train_data,train_label)

In [10]:
test_pool = Pool(test_data)

In [11]:
from sklearn import metrics

In [15]:
from sklearn.metrics import accuracy_score
model = CatBoostClassifier(
                           learning_rate=0.01,
                           l2_leaf_reg=3,
                           n_estimators = 7000,
                           loss_function='MultiClass',
                           task_type='GPU',
                           verbose=True)
# train the model
from sklearn.model_selection import cross_val_score

model.fit(train_pool)
# make the prediction using the resulting model
preds_class = model.predict(test_pool, prediction_type='Class')
#print(accuracy_score(test_label,preds_class))
print('confusion matrix')
print(metrics.confusion_matrix(test_label, preds_class))
print('classification report')
print(metrics.classification_report(test_label, preds_class))
print('Accuracy : %f' % (metrics.accuracy_score(test_label, preds_class)))

流式输出内容被截断，只能显示最后 5000 行内容。
2025:	learn: 0.2984413	total: 16s	remaining: 39.4s
2026:	learn: 0.2982381	total: 16.1s	remaining: 39.4s
2027:	learn: 0.2981623	total: 16.1s	remaining: 39.4s
2028:	learn: 0.2980607	total: 16.1s	remaining: 39.4s
2029:	learn: 0.2980220	total: 16.1s	remaining: 39.4s
2030:	learn: 0.2978978	total: 16.1s	remaining: 39.3s
2031:	learn: 0.2977539	total: 16.1s	remaining: 39.3s
2032:	learn: 0.2976212	total: 16.1s	remaining: 39.3s
2033:	learn: 0.2975420	total: 16.1s	remaining: 39.3s
2034:	learn: 0.2974183	total: 16.1s	remaining: 39.3s
2035:	learn: 0.2972434	total: 16.1s	remaining: 39.3s
2036:	learn: 0.2970975	total: 16.1s	remaining: 39.3s
2037:	learn: 0.2969149	total: 16.1s	remaining: 39.3s
2038:	learn: 0.2968481	total: 16.1s	remaining: 39.3s
2039:	learn: 0.2967371	total: 16.1s	remaining: 39.3s
2040:	learn: 0.2966428	total: 16.2s	remaining: 39.3s
2041:	learn: 0.2966010	total: 16.2s	remaining: 39.2s
2042:	learn: 0.2964618	total: 16.2s	remaining: 39.2s
2043:	learn: 0.296286

In [ ]:
a = pd.DataFrame(preds_class)

In [ ]:
a.to_csv('./aaa.csv')

##加入smote

In [ ]:
print(Counter(train_label))

Counter({7: 928, 3: 751, 2: 710, 5: 482, 1: 194, 4: 166, 6: 114})


In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
sm = SMOTE(random_state=123)
X_train_sm, y_train_sm = sm.fit_sample(train_data,train_label)

train_data_sm = pd.DataFrame(X_train_sm)
train_label_sm = pd.DataFrame(y_train_sm)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [ ]:
train_pool_sm = Pool(train_data_sm,train_label_sm)

In [ ]:
from sklearn.metrics import accuracy_score
model = CatBoostClassifier(
                           learning_rate=0.01,
                           l2_leaf_reg=3,
                           n_estimators = 8000,
                           loss_function='MultiClass',
                           verbose=True)
# train the model
model.fit(train_pool_sm)
# make the prediction using the resulting model
preds_class = model.predict(test_pool, prediction_type='Class')
print('confusion matrix')
print(metrics.confusion_matrix(test_label, preds_class))
print('classification report')
print(metrics.classification_report(test_label, preds_class))
print('Accuracy : %f' % (metrics.accuracy_score(test_label, preds_class)))

流式输出内容被截断，只能显示最后 5000 行内容。
3001:	learn: 0.1618922	total: 45.4s	remaining: 1m 15s
3002:	learn: 0.1618710	total: 45.5s	remaining: 1m 15s
3003:	learn: 0.1618179	total: 45.5s	remaining: 1m 15s
3004:	learn: 0.1617903	total: 45.5s	remaining: 1m 15s
3005:	learn: 0.1617322	total: 45.5s	remaining: 1m 15s
3006:	learn: 0.1616473	total: 45.5s	remaining: 1m 15s
3007:	learn: 0.1615991	total: 45.5s	remaining: 1m 15s
3008:	learn: 0.1615654	total: 45.5s	remaining: 1m 15s
3009:	learn: 0.1614731	total: 45.6s	remaining: 1m 15s
3010:	learn: 0.1614189	total: 45.6s	remaining: 1m 15s
3011:	learn: 0.1613493	total: 45.6s	remaining: 1m 15s
3012:	learn: 0.1612830	total: 45.6s	remaining: 1m 15s
3013:	learn: 0.1612346	total: 45.6s	remaining: 1m 15s
3014:	learn: 0.1611772	total: 45.6s	remaining: 1m 15s
3015:	learn: 0.1611478	total: 45.6s	remaining: 1m 15s
3016:	learn: 0.1611062	total: 45.7s	remaining: 1m 15s
3017:	learn: 0.1610479	total: 45.7s	remaining: 1m 15s
3018:	learn: 0.1609602	total: 45.7s	remaining: 1m 15s
3

CatBoostError: ignored

#美国油田

##引入数据

In [ ]:
! git clone https://github.com/seg/tutorials-2016.git

fatal: destination path 'tutorials-2016' already exists and is not an empty directory.


In [ ]:
%matplotlib inline
#%matplotlib inline 可以在Ipython编译器里直接使用，功能是可以内嵌绘图，并且可以省略掉plt.show()这一步。
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pandas import set_option
set_option("display.max_rows", 10)#设置要显示的默认行数，显示的最大行数是10
pd.options.mode.chained_assignment = None #为了在增加列表行数的时候防止出现setting with copy warning
filename = 'facies_vectors.csv'
training_data_us = pd.read_csv('/content/tutorials-2016/1610_Facies_classification/training_data.csv')
training_data_us

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.450,0.664,9.900,11.915,4.600,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.260,0.661,14.200,12.565,4.100,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.050,0.658,14.800,13.050,3.600,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.100,0.655,13.900,13.115,3.500,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.580,0.647,13.500,13.300,3.400,1,0.915
...,...,...,...,...,...,...,...,...,...,...,...
3227,5,C LM,CHURCHMAN BIBLE,3120.5,46.719,0.947,1.828,7.254,3.617,2,0.685
3228,5,C LM,CHURCHMAN BIBLE,3121.0,44.563,0.953,2.241,8.013,3.344,2,0.677
3229,5,C LM,CHURCHMAN BIBLE,3121.5,49.719,0.964,2.925,8.013,3.190,2,0.669
3230,5,C LM,CHURCHMAN BIBLE,3122.0,51.469,0.965,3.083,7.708,3.152,2,0.661


In [ ]:
blind_data = training_data_us[training_data_us['Well Name'] == 'SHANKLE']
training_data_us = training_data_us[training_data_us['Well Name'] != 'SHANKLE']

In [ ]:
training_data_us['Well Name'] = training_data_us['Well Name'].astype('category')
training_data_us['Formation'] = training_data_us['Formation'].astype('category')
training_data_us['Well Name'].unique()

['SHRIMPLIN', 'LUKE G U', 'CROSS H CATTLE', 'NOLAN', 'Recruit F9', 'NEWBY', 'CHURCHMAN BIBLE']
Categories (7, object): ['SHRIMPLIN', 'LUKE G U', 'CROSS H CATTLE', 'NOLAN', 'Recruit F9', 'NEWBY',
                         'CHURCHMAN BIBLE']

In [ ]:
correct_facies_labels = training_data_us['Facies'].values

feature_vectors = training_data_us.drop(['Formation', 'Well Name', 'Depth','Facies'], axis=1)
feature_vectors.describe()

,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
count,2783.000000,2783.000000,2783.000000,2783.000000,2783.000000,2783.000000,2783.000000
mean,66.249445,0.644637,3.754963,13.118929,3.805693,1.523895,0.523057
std,31.610849,0.241922,5.045916,7.389665,0.894118,0.499518,0.287499
min,13.250000,-0.025949,-21.832000,0.550000,0.200000,1.000000,0.010000
25%,46.081500,0.497000,1.300000,8.165000,3.200000,1.000000,0.276500
50%,65.541000,0.627000,3.581000,11.900000,3.600000,2.000000,0.529000
75%,80.714000,0.812323,6.500000,16.144000,4.400000,2.000000,0.771500
max,361.150000,1.480000,18.500000,84.400000,8.094000,2.000000,1.000000


In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        scaled_features, correct_facies_labels, test_size=0.2, random_state=42)

##使用smote

In [ ]:
from collections import Counter
print(Counter(y_train))

Counter({2: 519, 3: 414, 8: 370, 6: 319, 4: 144, 5: 138, 1: 131, 9: 127, 7: 64})


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=123,sampling_strategy={
                                              1:400, 5:400,4:300,9:300,7:300
})
X_train_sm, y_train_sm = sm.fit_sample(X_train,y_train)

X_train = pd.DataFrame(X_train_sm)
y_train = pd.DataFrame(y_train_sm)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

##使用catboost

In [ ]:
train_pool_us = Pool(X_train,y_train)

In [ ]:
test_pool_us = Pool(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
model = CatBoostClassifier(
                           learning_rate=0.1,
                           l2_leaf_reg=3,
                           n_estimators = 3000,
                           loss_function='MultiClass',
                           verbose=True)
# train the model
model.fit(train_pool_us)
#make the prediction using the resulting model
preds_class = model.predict(test_pool_us, prediction_type='Class')
print(accuracy_score(y_test,preds_class))

0:	learn: 1.9909829	total: 15.6ms	remaining: 46.8s
1:	learn: 1.8520176	total: 32.1ms	remaining: 48.1s
2:	learn: 1.7489356	total: 47.5ms	remaining: 47.4s
3:	learn: 1.6621295	total: 62.5ms	remaining: 46.8s
4:	learn: 1.5841825	total: 78.3ms	remaining: 46.9s
5:	learn: 1.5157102	total: 93.9ms	remaining: 46.9s
6:	learn: 1.4567459	total: 109ms	remaining: 46.7s
7:	learn: 1.4014123	total: 125ms	remaining: 46.8s
8:	learn: 1.3597356	total: 141ms	remaining: 46.8s
9:	learn: 1.3172136	total: 157ms	remaining: 46.8s
10:	learn: 1.2824332	total: 172ms	remaining: 46.9s
11:	learn: 1.2468087	total: 188ms	remaining: 46.7s
12:	learn: 1.2169950	total: 202ms	remaining: 46.4s
13:	learn: 1.1889819	total: 220ms	remaining: 46.9s
14:	learn: 1.1701153	total: 225ms	remaining: 44.7s
15:	learn: 1.1481506	total: 240ms	remaining: 44.8s
16:	learn: 1.1264577	total: 255ms	remaining: 44.8s
17:	learn: 1.1077648	total: 271ms	remaining: 44.9s
18:	learn: 1.0883640	total: 286ms	remaining: 44.9s
19:	learn: 1.0704880	total: 302ms	r

##引入盲井

In [ ]:
blind_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
471,2,A1 SH,SHANKLE,2774.5,98.36,0.642,-0.1,18.685,2.9,1,1.000
472,2,A1 SH,SHANKLE,2775.0,97.57,0.631,7.9,16.745,3.2,1,0.984
473,2,A1 SH,SHANKLE,2775.5,98.41,0.615,12.8,14.105,3.2,1,0.968
474,2,A1 SH,SHANKLE,2776.0,85.92,0.597,13.0,13.385,3.4,1,0.952
475,2,A1 SH,SHANKLE,2776.5,83.16,0.592,12.3,13.345,3.4,1,0.935
...,...,...,...,...,...,...,...,...,...,...,...
915,8,C LM,SHANKLE,3006.0,32.84,1.120,-2.2,3.455,5.1,2,0.060
916,8,C LM,SHANKLE,3006.5,25.16,1.112,-1.6,2.890,4.8,2,0.045
917,8,C LM,SHANKLE,3007.0,25.16,1.112,-1.6,2.890,4.8,2,0.030
918,4,C LM,SHANKLE,3007.5,26.22,1.092,-0.4,3.400,4.5,2,0.030


In [ ]:
blind_data['Well Name'] = blind_data['Well Name'].astype('category')
blind_data['Formation'] = blind_data['Formation'].astype('category')

In [ ]:
blind_data

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
471,2,A1 SH,SHANKLE,2774.5,98.36,0.642,-0.1,18.685,2.9,1,1.000
472,2,A1 SH,SHANKLE,2775.0,97.57,0.631,7.9,16.745,3.2,1,0.984
473,2,A1 SH,SHANKLE,2775.5,98.41,0.615,12.8,14.105,3.2,1,0.968
474,2,A1 SH,SHANKLE,2776.0,85.92,0.597,13.0,13.385,3.4,1,0.952
475,2,A1 SH,SHANKLE,2776.5,83.16,0.592,12.3,13.345,3.4,1,0.935
...,...,...,...,...,...,...,...,...,...,...,...
915,8,C LM,SHANKLE,3006.0,32.84,1.120,-2.2,3.455,5.1,2,0.060
916,8,C LM,SHANKLE,3006.5,25.16,1.112,-1.6,2.890,4.8,2,0.045
917,8,C LM,SHANKLE,3007.0,25.16,1.112,-1.6,2.890,4.8,2,0.030
918,4,C LM,SHANKLE,3007.5,26.22,1.092,-0.4,3.400,4.5,2,0.030


In [ ]:
blind_label = blind_data['Facies']

In [ ]:
blind_data = blind_data.drop(['Formation', 'Well Name', 'Depth','Facies'], axis=1)

In [ ]:
blind_data.reset_index(drop=True, inplace=True)

In [ ]:
blind_data

,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,98.36,0.642,-0.1,18.685,2.9,1,1.000
1,97.57,0.631,7.9,16.745,3.2,1,0.984
2,98.41,0.615,12.8,14.105,3.2,1,0.968
3,85.92,0.597,13.0,13.385,3.4,1,0.952
4,83.16,0.592,12.3,13.345,3.4,1,0.935
...,...,...,...,...,...,...,...
444,32.84,1.120,-2.2,3.455,5.1,2,0.060
445,25.16,1.112,-1.6,2.890,4.8,2,0.045
446,25.16,1.112,-1.6,2.890,4.8,2,0.030
447,26.22,1.092,-0.4,3.400,4.5,2,0.030


In [ ]:
blind_pool = Pool(blind_data)

In [ ]:
preds_class = model.predict(blind_pool, prediction_type='Class')
print(accuracy_score(blind_label,preds_class))

0.04899777282850779
